In [1]:
#Data Data Cleaning and Exploratory Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:.2f}'.format #para os números aparecerem com duas casas decimais
pd.set_option('display.max_columns', 50) #mostra 50 colunas ou o número desejado
pd.set_option('future.no_silent_downcasting', True)

dados = pd.read_excel("dados-2016-2021.xlsx", decimal=',')
dados.shape

#salvando os dados de 2021
dados21 = dados[dados['ANO_APOLICE'] == 2021]
dados21.shape

#Cada uma das 212839 linhas, é uma apólice contratada por um produtor. 

dados21.head

#cópia de segurança (copy of our database), vamos trabalhar com uma cópia dos dados, não com o conjunto original
seg = dados21.copy()

seg.head()

seg.tail()

seg.sample(10)

seg.columns #mostra as variáveis que temos no conjunto de dados

seg.info() #algumas variáveis, como DT_FIM_VIGENCIA vão precisar ter o seu dtype alterado. 

#FILTRANDO LINHAS #fazendo apenas consultas na base
#aqui vamos usar principalmente o método iloc
#iloc: que é utilizado para selecionar linhas e colunas, a partir de sua posição inteira
#loc: e não a partir dos rótulos ou nomes das linhas/colunas 

seg.iloc[:20, :10] #os dois pontos indicam toda a coluna

seg['NM_CULTURA_GLOBAL'].unique() #verificando quais tipos de culturas


seg['NM_MUNICIPIO_PROPRIEDADE'].unique() #verificando quantos municipios

seg['SG_UF_PROPRIEDADE'].unique() #verificando os estados

#selecionando a partir do nome da coluna
seg.loc[:, ['NR_AREA_TOTAL', 'NR_PRODUTIVIDADE_ESTIMADA', 'VL_LIMITE_GARANTIA', 
            'VL_PREMIO_LIQUIDO', 'VL_SUBVENCAO_FEDERAL', 'NR_APOLICE', 'NM_CULTURA_GLOBAL']]
 

#selecionando o estado do PR
seg_pr = seg.loc[seg['SG_UF_PROPRIEDADE'] == 'PR'] 

#principais culturas seguradas do Estado do PR
seg_pr['NM_CULTURA_GLOBAL'].unique()

#salvando os dados do meu município
seg_maringa = seg.loc[seg['NM_MUNICIPIO_PROPRIEDADE'] == 'Maringá'] 

#principais safras seguradas do meu municipio
seg_maringa['NM_CULTURA_GLOBAL'].unique() 

#poderíamos usar o query também
seg.query('(NM_MUNICIPIO_PROPRIEDADE == "Maringá") and (NM_CULTURA_GLOBAL == "Soja")')

In [ ]:
#Resumindo os dados

seg.describe()

In [ ]:
medidas_resumo1 = seg[['VL_PREMIO_LIQUIDO', 'VL_SUBVENCAO_FEDERAL']].quantile([0.25, 0.5, 0.75])
medidas_resumo1

In [ ]:
medidas_resumo2 = seg[['VL_PREMIO_LIQUIDO', 'VL_SUBVENCAO_FEDERAL', 'VL_LIMITE_GARANTIA', 'NR_AREA_TOTAL']].agg(['mean', 'min', 'max'])
medidas_resumo2

In [ ]:
#seguradoras
seg['NM_RAZAO_SOCIAL'].unique()

In [ ]:
#apolices por seguradora, Normalize = True p/ porcentagem
apolices_por_seguradora = seg['NM_RAZAO_SOCIAL'].value_counts(normalize=True, ascending=True) 
apolices_por_seguradora

In [ ]:
apolices_por_seguradora.plot.barh()
plt.title("Apolices por seguradora (%)")

In [ ]:
#LIMPEZA E ORGANIZAÇÃO DOS DADOS

In [ ]:
#verificar se existe algum valor faltante, sem o sum retorna true e false
seg.isna().sum()

In [ ]:
#ações possíveis para tratamento de informações faltantes (caso seja necessario)
seg = seg.fillna(0) #substitui faltantes por algum valor



In [ ]:
#substituir faltantes de uma variável pela média da variável
seg['VL_PREMIO_LIQUIDO'] = seg['VL_PREMIO_LIQUIDO'].fillna(seg['VL_PREMIO_LIQUIDO'].mean())

In [ ]:
#aqui temos alguns caracteres como '-', por isso vamos retirá-los no 
seg['NR_ANIMAL'].unique()

In [ ]:
#substituir caracteres como '.', '-', '...', etc
#salvando em um novo objeto, para manter os dados em seg originais
df = seg.replace(['-', '...', 'X'], 0)
df.dropna(inplace=True, how='all') #apagar ,linhas, caso ainda exista alguma com valores faltantes. 


In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
#Sera criada uma nova coluna, com a duração das apolices em dias. DT_FIM_VIGENCIA, está como objeto e precisa ser alterada. 

df['DT_FIM_VIGENCIA'] = pd.to_datetime(df['DT_FIM_VIGENCIA'], errors = 'coerce')

#criar uma nova coluna com a duração das apólices
df['duracao'] = (df['DT_FIM_VIGENCIA'] - df['DT_INICIO_VIGENCIA']).dt.days

#ordenar os valores da coluna nova
df.sort_values('duracao', ascending=False)

In [ ]:
df.info()

In [ ]:
#algumas colunas serão retiradas
retirar = ['CD_PROCESSO_SUSEP', 'NR_PROPOSTA', 'ID_PROPOSTA', 'DT_PROPOSTA', 'DT_INICIO_VIGENCIA', 
           'DT_FIM_VIGENCIA', 'NM_SEGURADO', 'NR_DOCUMENTO_SEGURADO', 'LATITUDE', 'NR_GRAU_LAT', 'NR_MIN_LAT', 
          'NR_SEG_LAT', 'LONGITUDE', 'NR_GRAU_LONG', 'NR_MIN_LONG', 'NR_SEG_LONG', 'NR_DECIMAL_LATITUDE', 
           'NR_DECIMAL_LONGITUDE', 'DT_APOLICE', 'ANO_APOLICE']
df = df.drop(retirar, axis = 1)

In [ ]:
#necessario retirar os catacteres a mais
df['EVENTO_PREPONDERANTE'].unique() 

In [ ]:
df['EVENTO_PREPONDERANTE'] = df['EVENTO_PREPONDERANTE'].str.split().str.join('  ') #separa as strings e junta elas apenas onde ficaram com 2 espaços
df['EVENTO_PREPONDERANTE'] = df['EVENTO_PREPONDERANTE'].replace([np.nan], '0') #OBS: existem vários tipos de 

In [ ]:
df['EVENTO_PREPONDERANTE'].unique()

In [ ]:
df.shape

In [ ]:
#retirar espaços em branco (caso haja algum)
df['NM_RAZAO_SOCIAL'] = df['NM_RAZAO_SOCIAL'].str.strip()
df['NM_CULTURA_GLOBAL'] = df['NM_CULTURA_GLOBAL'].str.strip()

In [ ]:
#USO DO GROUPBY
df.groupby('SG_UF_PROPRIEDADE')['VL_PREMIO_LIQUIDO'].sum().sort_values(ascending=False) #agrupando a soma dos prêmios líquidos por Estado

In [ ]:
premio_estado =  df.groupby('SG_UF_PROPRIEDADE')['VL_PREMIO_LIQUIDO'].sum().sort_values(ascending=True)
premio_estado.plot.barh()
plt.title("Valores segurados por Estado (Bilhões)")

In [ ]:
#CALCULANDO O ÍNDICE DE SINISTRALIDADE
#O índice mede (em %) a participação do valor total de indenizações pagas aos produtores,
#em relação ao valor total do prêmio arrecadado pelas seguradoras. 65% é entendido como valor limite aceitável. 
#Esse índice permite inferencias sobre o equilíbrio atuarial das operações de seguro rural. 

sinistralidade = (df['VALOR_INDENIZAÇÃO'].sum()) / (df['VL_PREMIO_LIQUIDO'].sum()) * 100 
print("Índice de Sinistralidade" + " é " + str(sinistralidade) + " %")

In [ ]:
#alterando o nome das colunas, lembrando que todas precisam estar na ordem que aparecem no dataframe
df.columns = ['seguradora', 'municipio', 'uf', 'tipo_prod', 'cultura', 
             'area', 'animal', 'prod_est', 'prod_seg', 'nivel_cob', 'total_seg', 'premio',
             'taxa', 'subvencao', 'apolice', 'cod_mun', 'indenizacao', 'evento', 'duracao'] 

In [ ]:
#novos nomes; Dtype ok
df.info()

In [ ]:
#transformando mun em inteiro, pois é um código
df['cod_mun'] = df['cod_mun'].astype(int)
df['indenizacao'] = df['indenizacao'].astype(float)

In [ ]:
#criando uma coluna com a sinistralidade
df['sinistralidade'] = df['indenizacao'] / df['premio']
df.sample(10)

In [ ]:
#variável de indicação: 0 = não houve sinistro; 1 = houve sinistro a partir da variável evento
#caso 'evento' seja 0 = não houve sisnitro; se 'evento' = SECA, CHUVA EXCESSIVA, etc., existiu sinistro
df['sinistro'] = np.where(df['evento'].str.isnumeric(), 0, 1)
df.sample(10)

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
#Reorganizando as variáveis para melhor visualização usando .loc
df = df.loc[:, ['apolice', 'cod_mun', 'municipio', 'uf', 'tipo_prod','seguradora', 'cultura', 'area', 'animal', 
                'duracao', 'prod_est', 'prod_seg', 'total_seg', 'premio', 'taxa', 'subvencao',
               'indenizacao', 'evento', 'sinistro', 'sinistralidade']]

In [ ]:
df.info()

In [ ]:
#ascending = True, mostra do menor para o maior.
df.sort_values('premio') 

In [ ]:
df.columns

In [ ]:
#selecionando algumas colunas e o valor de subvenção
df[['municipio', 'uf', 'cultura', 'subvencao']].sort_values('subvencao', ascending=False)

In [ ]:
df.shape #mesmo número de linhas da base SEG

In [ ]:
#Salvando os dados limpos como xlsx (index = false para não salvar o df como uma coluna do df atual)
df.to_excel("seguro-rural-limpo-2016-2021.xlsx", index = False)

In [ ]:
#caso quisermos salvar como csv
#df.to_excel("seguro-rural-2021.csv", index=False, encoding='latin1')

In [ ]:
#verificar se os dados serão lidos corretamente
#com os dados limpos, vamos fazer a análise Exploratória
df = pd.read_excel("seguro-rural-limpo-2016-2021.xlsx")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
#resumo estatístico apenas de algumas variáveis, seleção usando iloc (números das linhas e colunas)
df.iloc[:, 5:].describe()

In [ ]:
#Existem dados duplicados? 
df.loc[df.duplicated()]

In [ ]:
#podemos usar para um subconjunto dos dados
df.loc[df.duplicated(subset=['apolice'])]

In [ ]:
#verificar dados únicos para todo o conjunto de dados
df.nunique()

In [ ]:
#ocorrencias unicas de evento
df['evento'].unique()

In [ ]:
#trocando os nomes das ocorrencias de evento para melhorar a plotagem
#o argumento da função é um dicionário
df['evento'].replace({'0' : 'nenhum',
                      'SECA' : 'seca', 
                      'GEADA': 'geada',
                      'GRANIZO': 'granizo',
                      'VENTOS  FORTES/FRIOS': 'ventos',
                      'CHUVA  EXCESSIVA': 'chuva',
                      'INCÊNDIO': 'incendio',
                      'DEMAIS  CAUSAS': 'demais causas',
                      'VARIAÇÃO  EXCESSIVA  DE  TEMPERATURA': 'temperatura',
                      'RAIO': 'raio',
                      'MORTE': 'morte',
                      'INUNDAÇÃO/TROMBA  D´ÁGUA': 'inundacao',
                      'VARIAÇÃO  DE  PREÇO': 'var. preco',
                      'DOENÇAS  E  PRAGAS': 'doenças',
                      'QUEDA  DE  PARREIRAL': 'queda parreiral',
                      'PERDA  DE  QUALIDADE': 'perda qualidade',
                      'REPLANTIO': 'replantio'}, inplace=True)

In [ ]:
#ocorrencias unicas de evento
df['evento'].unique()

In [ ]:
df['tipo_prod'].unique()

In [ ]:
#trocar de maíusculas pra minúsculas
df['tipo_prod'].replace({ 'CUSTEIO': 'custeio',
                          'PRODUTIVIDADE': 'produtividade',
                          'PECUÁRIO': 'pecuario',
                          'FLORESTAS': 'florestas'}, inplace=True)

In [ ]:
#algumas consultas, por exemplo, quais foram as apolices de uva? 
#verificando as culturas 
df['cultura'].unique()

In [ ]:
#apolices de uva 
df[df['cultura'] == 'Uva']

In [ ]:
#organizando os valores pelo menor premio
df.sort_values(by='premio').head(10)

In [ ]:
#ordenando as apolices de forma decrescente de prêmio
#mostrando apenas algumas variáveis
df.loc[:, ['municipio', 'uf', 'cultura', 'premio']].sort_values(by='premio', ascending=False).head(10)
#entre as apolices com 10 maiores valores de premio, 7 são de soja. 

In [ ]:
#COMEÇANDO OS GRÁFICOS
#podemos usar: 
#histogramas, kde (kernel density estimation) gráficos para estimar as distribuições de probabilidade, boxplots, para variáveis QUANTITATIVAS
#gráficos de colunas, barras, linhas, pizza, para variáveis QUALITATIVAS
pd.DataFrame(df['cultura'].value_counts(ascending=False))

In [ ]:
#quantidade de apolices por cultura
df['cultura'].value_counts(ascending=True).plot.barh()

In [ ]:
#mostrando apenas as 20 culturas mais seguradas
culturas = df['cultura'].value_counts(ascending=True)
culturas.tail(20).plot.barh()

In [ ]:
#definindo um objeto ax para adicionar mais elementos ao gráfico
ax = df['cultura'].value_counts(ascending=True)\
    .tail(10)\
    .plot(kind='barh', title='10 culturas com mais apólices', color = 'purple')
ax.set_xlabel('Cultura')
ax.set_ylabel('Quantidade apólices')



In [ ]:
#a porcentagem de apólices de soja é 46,9%
ax = df['cultura'].value_counts()\
    .plot(kind='pie', autopct='%1.1f%%',title='10 culturas com mais apólices (%)')
ax.set_xlabel(' ')
ax.set_ylabel(' ')

In [ ]:
#caso queiramos considerar apenas as 10 principais culturas
#vamos perder alguma informação sobre as porcentagens
ax = df['cultura'].value_counts()\
    .head(10)\
    .plot(kind='pie', autopct='%1.1f%%',title='10 culturas com mais apólices (%)')
ax.set_xlabel(' ')
ax.set_ylabel(' ')

In [ ]:
#afim de diminuir a quantidade de categorias
#vamos categorizar as culturas em classificações generalizantes
df['cultura'].unique()

In [ ]:
#classificando os tipos de culturas
df['tipo_cultura'] = df['cultura'].replace({ 'Milho 2ª safra': 'cereais',
                                             'Uva': 'frutas', 
                                             'Tomate': 'frutas', 
                                             'Feijão 1ª safra': 'leguminosas', 
                                             'Sorgo': 'cereais',
                                             'Café': 'perene', 
                                             'Trigo': 'cereais', 
                                             'Feijão 2ª safra':'leguminosas' , 
                                             'Milho 1ª safra':'cereais', 
                                             'Repolho':'vegetais',
                                             'Tangerina':'frutas', 
                                             'Atemoia':'frutas', 
                                             'Pecuário':'pecuario', 
                                             'Floresta':'floresta', 
                                             'Pepino':'vegetais',
                                             'Cana-de-açúcar':'acucar', 
                                             'Pimentão':'frutas', 
                                             'Soja':'cereais', 
                                             'Abobrinha':'vegetais', 
                                             'Alho':'perene',
                                             'Berinjela':'frutas', 
                                             'Beterraba':'legume', 
                                             'Canola':'vegetais', 
                                             'Melancia':'frutas', 
                                             'Abóbora':'vegetais',
                                             'Batata':'legume', 
                                             'Aveia':'cereais',
                                             'Cebola':'legume', 
                                             'Abacate':'frutas', 
                                             'Couve-flor':'vegetais', 
                                             'Laranja':'frutas',
                                             'Triticale':'cereais', 
                                             'Cenoura':'legume', 
                                             'Goiaba':'frutas', 
                                             'Cevada':'cereais', 
                                             'Banana':'frutas', 
                                             'Ervilha':'leguminosas',
                                             'Pêssego':'frutas', 
                                             'Ameixa':'frutas', 
                                             'Amendoim':'leguminosas', 
                                             'Mandioca':'legume', 
                                             'Maracujá':'frutas',
                                             'Brócolis':'vegetais', 
                                             'Alface':'vegetais', 
                                             'Caqui':'frutas', 
                                             'Nectarina':'frutas',
                                             'Mamão':'frutas', 
                                             'Kiwi':'frutas',
                                             'Vagem':'leguminosas', 
                                             'Maçã':'frutas', 
                                             'Arroz':'cereais',
                                             'Abacaxi':'frutas', 
                                             'Lichia':'frutas',
                                             'Manga':'frutas', 
                                             'Morango':'frutas',
                                             'Chuchu':'legume', 
                                             'Pêra':'frutas', 
                                             'Limão':'frutas',
                                             'Algodão':'perene',
                                             'Girassol':'cereais',
                                             'Figo':'frutas', 
                                             'Cacau':'frutas',
                                             'Melão':'frutas',
                                             'Pastagem':'pastagem'
    
})

In [ ]:
df['tipo_cultura'].unique()

In [ ]:
#colocando no gráfico a nova visualização
#valores totais
ax = df['tipo_cultura'].value_counts(ascending=True)\
    .plot(kind='barh', title='10 tipos de culturas com mais apólices', color = 'purple')
ax.set_xlabel('Quantidade Apólices')
ax.set_ylabel('Tipo Cultura')

In [ ]:
#valores em porcentagem
ax = df['tipo_cultura'].value_counts(normalize=True, ascending=True)\
    .plot(kind='barh', title='10 tipos de culturas com mais apólices', color = 'purple')
ax.set_xlabel('Quantidade Apólices')
ax.set_ylabel('Tipo Cultura')

In [ ]:
#colocando no gráfico a nova visualização
ax = df['tipo_cultura'].value_counts()\
    .plot(kind='pie', autopct='%1.1f%%',title='10 culturas com mais apólices (%)')
ax.set_xlabel(' ')
ax.set_ylabel(' ')

In [ ]:
#reoganizando as colunas no data frame
df = df.loc[:,['apolice', 'cod_mun', 'municipio', 'uf','seguradora', 'tipo_prod','tipo_cultura', 
            'cultura', 'area', 'animal', 'duracao', 'prod_est', 'prod_seg', 'total_seg', 'premio',
            'taxa', 'subvencao', 'indenizacao', 'evento', 'sinistro', 'sinistralidade']]

In [ ]:
#algumas variáveis quantitativas
#histograma
ax = df['premio'].plot(kind='hist', bins=20)
ax.set_ylabel('Número de Apólices')
ax.set_xlabel('Valor Prêmio (R$mil)')

#OBS: a maior parte das apólices são de pequenos valores segurados. 

In [ ]:
#olhando as apolices com valores maiores do que R$ 40 mil
premio = df[df['premio'] > 40000]
ax = premio['premio'].plot(kind='hist', bins=20)


In [ ]:
#olhando as apolices com valores maiores do que R$ 400 mil
premio = df[df['premio'] > 400000]
ax = premio['premio'].plot(kind='hist', bins=20)

#Conforme aumentamos o valor das apólices, podemos verificar que a ocorrência de apólices diminui
#ou seja, quanto maior o valor, temos menos apólices

In [ ]:
#o gráfico da distribuição feita no histograma, mas suavizada
ax = df['premio'].plot(kind='kde')

In [ ]:
#com um box plot, podemos ver isso melhor
sns.boxplot(x=df['premio'], )
#a quantidade de outliers é alta, ou seja temos vários valores
#que não estão presentes na maior quantidade de ocorrências dos dados.

In [ ]:
#explorando os outliers SEPARANDO OS DADOS EM "VALORES BAIXOS" E "VALORES ALTOS"
#o valor de 22598.85 é referente ao 3º quartil da variável premio
#ou seja, 75% dos dados são valores de apólices abaixo de R$ 22598.85
premio = df[df['premio'] > 22598.85]
premio.shape #são 53210 ocorrências de valores maiores de 22598.85, lembrando que o valor máximo é quase 900000.00

In [ ]:
#colocando no gráfico as principais culturas com "valores altos"(acima de 22598.85 até 890000) 
ax = premio['cultura'].value_counts()\
    .plot(kind='pie', autopct='%1.1f%%',title='Culturas (%)')
ax.set_xlabel('Valores Maiores que R$ 22598,85')
ax.set_ylabel(' ')

#das 53 mil ocorrências de "valores altos" 53.8% delas, ainda é de soja. 

In [ ]:
#distribuição dos valores de prêmio maiores do que 22598.85
premio_alto = df[df['premio'] > 22598.85]#são 53210 ocorrências de valores maiores de 22598.85

sns.boxplot(x=premio_alto['premio'])

In [ ]:
#explorando os outliers 
#o valor de 22598.85 é referente ao 3º quartil da variável premio
#ou seja, 75% dos dados são valores de apólices abaixo de R$ 22598.85
premio = df[df['premio'] < 22598.85]
premio.shape #são 159629 ocorrências de valores menores de 22598.85

In [ ]:
#colocando no gráfico as principais culturas com "valores baixos"(abaixo de 22598.85) 
ax = premio['cultura'].value_counts()\
    .plot(kind='pie', autopct='%1.1f%%',title='Culturas (%)')
ax.set_xlabel('Valores Menores que R$ 22598.85')
ax.set_ylabel(' ')

#das 159629 mil ocorrências de "valores baixos" , 44.6% é de soja

In [ ]:
#distribuição dos valores de prêmio menores do que 22598.85
#são 159629 ocorrências de valores menores de 22598.85
premio_baixo = df[df['premio'] < 22598.85]
premio_baixo['premio'].describe()
sns.boxplot(x=premio_baixo['premio'])

In [ ]:
print(f'A soma de todas as apólices com valores baixos é: R$ {premio_baixo['premio'].sum():.2f}')

print(f'A soma de todas as apólices com valores altos é: R$ {premio_alto['premio'].sum():.2f}')

print(f'Assim os valores baixos representam: {(premio_baixo['premio'].sum()/df['premio'].sum()*100):.2f}% do valor total das apólices')

print(f'Assim os valores altos representam: {(premio_alto['premio'].sum()/df['premio'].sum()*100):.2f}%  do valor total das apólices')

In [ ]:
#FAZENDO BOXPLOTS COM MAIS DE UMA VARIÁVEL QUANTITATIVA
num_cols = df.iloc[:, np.r_[10:18, 20]]
col_names = num_cols.columns
col_names

fig = plt.figure(figsize=(10,7))

for i in range(9):
    fig.tight_layout() #para formatar os gráficos proporcionalmente

    grid_col = "33{}".format(i+1) #argumento da função add_subplot para organizar os gráficos em grade

    ax = fig.add_subplot(int(grid_col))

    sns.boxplot(data=df, x=col_names[i], ax=ax)

    

In [ ]:
col_names

In [ ]:
#VERIFICANDO RELAÇÕES ENTRE VARIÁVEIS
#Diagramas de dispersão, mapas de calor 
#selecionando as colunas de interesse
df_corr = df.iloc[:, 8:]
df_corr.info() #somente variáveis numéricas

In [ ]:
plt.figure(figsize=(20,15)) #tamanho em polegadas
sns.heatmap(df_corr.corr(numeric_only=True), cmap="BrBG", annot = True)

In [ ]:
#premio x indenização
#FAZENDO GRÁFICOS DE DISPERSÃO COM MAIS DE UMA VARIÁVEL QUANTITATIVA
num_cols = df.iloc[:, np.r_[10:18, 20]]
col_names = num_cols.columns
col_names

fig = plt.figure(figsize=(10,7))

for i in range(9):
    fig.tight_layout() #para formatar os gráficos proporcionalmente

    grid_col = "33{}".format(i+1) #argumento da função add_subplot para organizar os gráficos em grade

    ax = fig.add_subplot(int(grid_col))

    sns.scatterplot(x=col_names[i],
                     y=col_names[i-1],
                    hue='tipo_prod',
                    data=df)


In [ ]:
#FAZENDO OUTROS USOS DO GROUP BY 

#fazendo a média de todas as variáveis numéricas por Estado
df_uf_media = df.iloc[:, 2:] #retirando a coluna cod_mun
df_uf_media.groupby('uf').mean(numeric_only=True).sort_values(by='premio', ascending=False) #em ordem crescente

#RESULTADO: 
#1ª Linha: A maior média de premios por Estado
#2ª Linha em diante: as maiores médias de premios por Estado em direção das menores médias. 

In [ ]:
#AGREGANDO por cultura
#mostrando soma, média e frequencia
#do premio, subvencao e indenização

df.groupby(['cultura'])[['premio', 'subvencao', 'indenizacao']]\
    .agg([('media', 'mean'), ('freq', 'count'), ('soma', 'sum')])\
        .reset_index()

In [ ]:
#agregando por tipo_cultura
df.groupby(['tipo_cultura'])[['premio', 'subvencao', 'indenizacao']]\
    .agg([('media', 'mean'), ('freq', 'count'), ('soma', 'sum')])\
        .reset_index()

In [ ]:
#agrupando por tipo cultura e tirando a média
df_g = df.groupby(['tipo_cultura'])[['premio', 'subvencao', 'indenizacao']].mean()
df_g

In [ ]:
#agrupando por tipo cultura e somando
df_g = df.groupby(['tipo_cultura'])[['premio', 'subvencao', 'indenizacao']].sum()
df_g

In [ ]:
#agrupando por cultura, somando e ordenando do maior premio pro menor
df_cultura = df.groupby(['cultura']).sum().sort_values(by='premio', ascending=False)
df_cultura 

In [ ]:
#algumas questões sobre os dados
#cria um dataframe com as médias das variáveis por Estado
df_uf = df.groupby(by=['uf']).mean(numeric_only=True)
df_uf = df_uf.reset_index() #para colocar um novo indice, o índice UF vira uma coluna
df_uf.head()

In [ ]:
plt.plot(df_uf['uf'], df_uf['premio'], marker='o')
plt.title('')
plt.xlabel('UF')
plt.ylabel('Prêmio (R$mil)')

In [ ]:
#organizando os prêmios médios por Estado, em ordem decrescente
ax = df.groupby('uf')['premio'] \
    .agg(['mean', 'count']) \
    .sort_values('mean', ascending=False)['mean']\
    .plot(kind='bar', figsize=(12,5), title='Prêmio medio', color='purple')
ax.set_xlabel('Prêmio medio')

In [ ]:
#agregando a indenização e premio médios por UF.
ax = df.groupby('uf')[['premio', 'indenizacao']].mean(numeric_only=True) \
    .plot(kind='bar', figsize=(12,5), title='Prêmio medio', color=['purple', 'green'])
ax.set_xlabel('Premio e Indenização media')

In [ ]:
#o mesmo gráfico, mas utilizando o plt.
df_uf = df.groupby('uf')[['premio', 'indenizacao']].mean(numeric_only=True)\
      .plot.bar(figsize=(10, 5), layout='constrained', color = ['purple', 'green'])
df_uf.set_xlabel('UF')
df_uf.set_ylabel('Valores em R$mil')



In [ ]:
#verificando as culturas mais seguradas Estado
for i in df['uf'].unique():
    print(i, df[df['uf']==i]['cultura'].value_counts().index[0])

In [ ]:
#usando a função lambda (função anonima? Ela serve pra fazer alguma alteração rápida)
#podemos alterar por exemplo algumas informações do dataframe 
#podemos usar essa função para colocar nomes minusculos, por exemplo: 
#df['minusculo'] = df['nome'].apply(lambda x:x.lower())

df.groupby('uf')['cultura'].apply(lambda x: x.value_counts().index[0])

In [ ]:
df1 = df.copy()

In [ ]:
#AGREGANDO AS INFORMAÇÕES POR MUNICÍPIO
#fazendo um novo dataframe 

df1 = df1.groupby(['municipio', 'uf']).aggregate({ 'apolice': 'count',
                                             'area': 'sum',#declarando as operações da função aggregate
                                             'animal': 'sum',
                                             'duracao': 'sum',
                                             'prod_est':'sum',
                                             'prod_seg':'sum',
                                             'total_seg':'sum',
                                             'premio':'sum',
                                             'taxa':'sum',
                                             'subvencao':'sum',
                                             'indenizacao': 'sum'
})

df1 = df1.reset_index() #índices sejam posição das linhas 

In [ ]:
df1.head(10).sort_values(by='premio', ascending=True)

In [ ]:
df1_maiores_premios = df1.groupby(['municipio', 'uf'])['premio'].sum().head(10).sort_values( ascending=True)

fig, ax = plt.subplots(figsize=(10,5))
df1_maiores_premios.plot.barh()
ax.set_xlabel('Premio R$milhões')
ax.set_ylabel('UF, Municipio')
ax.set_title('Maiores premios, por municipio e Estado')

In [ ]:
#consulta de alguma cidade específica

df1[df1['municipio'] == 'Maringá']

In [ ]:
#agregando a informação por município, perdemos a informação das variáveis categóricas 
#assim, vamos usar a função lambda para verificar as culturas mais seguradas
df_cultura = pd.DataFrame(df.groupby(['municipio', 'uf'])['cultura'].apply(lambda x: x.value_counts().index[0]))
df_cultura = df_cultura.reset_index()

df2 = df1.merge(df_cultura)
df2.head(20).sort_values(by='premio')

In [ ]:
#salvar o novo DF
#df2.to_excel('seguro-rural-municipios-2021.xlsx')

In [ ]:




#function from: https://matplotlib.org/stable/users/explain/quick_start.html#quick-start
#função caso precise fazer as mesmas plotagens com dados diferentes
#data1 = seg[['VL_PREMIO_LIQUIDO', 'VL_SUBVENCAO_FEDERAL', 'NR_AREA_TOTAL']].quantile([0.25, 0.5, 0.75])
#data2 = seg[[

#def my_plotter(ax, data1, data2, param_dict):
#    """
#    A helper function to make a graph.
#    """
#    out = ax.plot(data1, data2, **param_dict)
#    return out


#data1, data2, data3, data4 = np.random.randn(4, 100)  # make 4 random data sets
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(5, 2.7))
#my_plotter(ax1, data1, data2, {'marker': 'x'})
#my_plotter(ax2, data3, data4, {'marker': 'o'})